# Unit 3 - Proportional, Integral and Derivative (PID) Controllers

# Objectves
- In previous Unit saw that proportional controllers are sometimes able to meet design requirements (they can increase the performance of a system)
- However proportional controllers typically have large steady state errors (which limits their usefulness)
- Integral control can eliminate steady state errors while still keeping the desired performance
- Derivative control can increase the performance of a system (reduce tracking errors)
- The main strength of PID control is that it allows for intuitive tuning (this is probably the reason for the popularity of PID control)
- 50\% of industrial controllers are PID controllers (Ogata 2010)

# What is a PID controller?
$$
\begin{align*}
u_K(t) &= K(t)*\varepsilon(t) \\
 &= \underbrace{K_p \varepsilon(t)}_{\substack{\text{Proportional} \\ \text{Control} \\ \text{Action}}} + \underbrace{K_i \int_0^{t}\varepsilon(\tau) d\tau}_{\substack{\text{Integral} \\ \text{Control} \\ \text{action}}}+ \underbrace{ K_d \frac{d}{dt} \varepsilon(t)}_{\substack{\text{Derivative} \\ \text{Control} \\ \text{action}}}
\end{align*}
$$

There are 3 "tuning knobs": $K_p$, $K_i$ and $K_d$. 
- Somehow we must pick a suitable value for each of these parameters. 
- By understanding how each parameter affects the closed-loop system we can tune the controller intuitively.
- Advantage: no need for a model!

In Laplace domain:
$$
\begin{align*}
U_K(s) &= K(s)\mathcal{E}(s) \\
 &= K_p \mathcal{E}(s)  + \frac{K_i}{s} \mathcal{E}(s)  + K_d \mathcal{E}(s)s
\end{align*}
$$

![PID feedback loop](PIDcontrolLoop.jpg "PID feedback loop")

# The Intuition Behind PID Controllers



# Start with Proportional Control

- Choose a value for $K_p$ to meet design requirements. If you have a model for the system, can use Root Locus Plots. 
- If design requirements cannot be met using just a proportional controller, add a derivative controller
- If design requirements are met using either P or PD controllers, then check the steady state error. If there is an error, add an integral controller. 

In [ ]:
# Integral Control

- 

# Steady State Errors

What is the steady state error for a proportional controller?

$$
\begin{align*}
\mathcal{E}(s) &= S(s)R(s) \\
&= \frac{1}{1 + P(s)K}R(s) \\
&= \frac{1}{1 + \frac{B(s)}{A(s)}K} \\
&= \frac{A(s)}{A(s)+ B(s)K}
\end{align*}
$$

# Ramp error

# Derivative Control



# The problem with derivative control - amplification of noise